# Spectractor Launcher

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Required packages 

In [20]:
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),   #defines the default image size
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [12]:
plt.rcParams.update({'figure.max_open_warning': 0}) # to see a large number of files

## Tools

In [13]:
sys.path.append("../common_tools/")
from common_notebook import *

init_notebook()

ccdproc version 1.3.0
bottleneck version 1.2.1
/pbs/throng/lsst/users/dagoret/GitHubatCC/SpectractorAnaAtm/ana_30may17


## Spectractor 

In [14]:
PATH_SPECTRACTOR='../../Spectractor'

In [15]:
sys.path.append(PATH_SPECTRACTOR)

In [16]:
from spectractor import *

# Configuration

## Input Directory

In [32]:
inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
MIN_IMGNUMBER=58
#MAX_IMGNUMBER=80  # temporary
MAX_IMGNUMBER=190
date="20170530"
object_name='HD111980'
tag = "reduc"
SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
NBIMGPERROW=2
subdir=os.path.basename(inputdir)

# Output directory

In [ ]:
dir_top_images="images_notebooks/"+theNotebook ### Top directory to save images
ensure_dir(dir_top_images) # create that directory
outputdir="./cut_fitsimages"
ensure_dir(outputdir)

In [7]:
infile_fullpath=os.path.join(PATH_SPECTRACTOR,infile)

In [8]:
sys.path.append(PATH_SPECTRACTOR)

In [9]:
from spectractor import *

# Logbook

In [28]:
file_logbook_csv='../common_info/ctiofulllogbook_jun2017_v3.csv'
file_logbook_xlsx='../common_info/ctiofulllogbook_jun2017_v3.xlsx'

In [29]:
#df=pd.read_csv(file_logbook_csv,index_col=0)
df=pd.read_excel(file_logbook_xlsx,index_col=0)

In [31]:
df.head()

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip
215,2017-05-30T21:17:00.652,data_30may17,1,bias,RG715,Ron400,1.181,0.0,12650,1.724,...,7.6,36,5.6,reduc_20170630_001.fits,NaN,NaN,NaN,NaN,NaN,NaN
147,2017-05-30T21:17:44.185,data_30may17,2,bias,RG715,Ron400,1.181,0.0,12650,1.724,...,7.6,36,5.6,reduc_20170630_002.fits,NaN,NaN,NaN,NaN,NaN,NaN
13,2017-05-30T21:18:27.708,data_30may17,3,bias,RG715,Ron400,1.181,0.0,12650,1.724,...,7.5,36,4.9,reduc_20170630_003.fits,NaN,NaN,NaN,NaN,NaN,NaN
45,2017-05-30T21:19:11.234,data_30may17,4,bias,RG715,Ron400,1.181,0.0,12650,1.724,...,7.5,36,4.7,reduc_20170630_004.fits,NaN,NaN,NaN,NaN,NaN,NaN
218,2017-05-30T21:19:54.772,data_30may17,5,bias,RG715,Ron400,1.181,0.0,12650,1.724,...,7.5,36,4.7,reduc_20170630_005.fits,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
all_obs=df.loc[((df["subdir"]==subdir) & (df["object"]==object_name)),:]

In [51]:
all_obs

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip


In [49]:
(df["subdir"]==subdir) & (df["object"]==object_name)

215    False
147    False
13     False
45     False
218    False
250    False
229    False
159    False
224    False
0      False
106    False
145    False
36     False
205    False
74     False
208    False
107    False
268    False
61     False
140    False
247    False
34     False
171    False
199    False
110    False
255    False
229    False
230    False
7      False
88     False
       ...  
178    False
217    False
274    False
16     False
228    False
260    False
81     False
152    False
15     False
8      False
59     False
12     False
85     False
49     False
7      False
148    False
9      False
73     False
81     False
34     False
100    False
70     False
32     False
55     False
92     False
121    False
93     False
22     False
61     False
87     False
dtype: bool

In [40]:
all_files=df.loc[df["subdir"]==subdir,['file']]

,file
215,reduc_20170630_001.fits
147,reduc_20170630_002.fits
13,reduc_20170630_003.fits
45,reduc_20170630_004.fits
218,reduc_20170630_005.fits
250,reduc_20170630_006.fits
229,reduc_20170630_007.fits
159,reduc_20170630_008.fits
224,reduc_20170630_009.fits
0,reduc_20170630_010.fits


In [35]:
?df.loc()

Type:        property
String form: <property object at 0x7f7ab9dbd368>
Docstring:  
Purely label-location based indexer for selection by label.

``.loc[]`` is primarily label based, but may also be used with a
boolean array.

Allowed inputs are:

- A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
  interpreted as a *label* of the index, and **never** as an
  integer position along the index).
- A list or array of labels, e.g. ``['a', 'b', 'c']``.
- A slice object with labels, e.g. ``'a':'f'`` (note that contrary
  to usual python slices, **both** the start and the stop are included!).
- A boolean array.
- A ``callable`` function with one argument (the calling Series, DataFrame
  or Panel) and that returns valid output for indexing (one of the above)

``.loc`` will raise a ``KeyError`` when the items are not found.

See more at :ref:`Selection by Label <indexing.label>`
